## Satisfação dos clientes do Banco Santader

### Problema: Prever a satisfação do cliente do banco santander em relação a sua experiência bancária

Dataset: https://www.kaggle.com/c/santander-customer-satisfaction/overview

Meta: Acurácia igual ou maior a <b>70%</b>

In [1]:
# Imports
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

# Import para alterar o número de colunas exibidas em dataframes do pandas
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
# Criando Spark Session para trabalhar com Data Frame
spSession = SparkSession.builder.master("local").appName("santander-customer-satisfaction").getOrCreate()

In [3]:
# Carregados dados do clientes (dataset criado no script de SMOTE)
clientesRDD = sc.textFile("data/clientes_smote.csv")

In [4]:
clientesRDD.count()

146025

In [5]:
# Removendo cabeçalho
header = clientesRDD.first()
clientesRDD2 = clientesRDD.filter(lambda line: line != header)

In [6]:
clientesRDD2.count()

146024

In [7]:
clientesRDD2.cache()

PythonRDD[5] at RDD at PythonRDD.scala:53

## Limpeza dos Dados

In [8]:
# Criando dataframe from RDD
# Define uma estrutura para o Row com base no header, pois eram muitas colunas para fazer manualmente
row = Row(*header.split(","))
def transformToRow(line):
    attList = line.split(",")
    
    for i in range(len(attList)):
        attList[i] = float(attList[i])
    
    # cria e retorna uma row com os valores convertidos para numéricos        
    return row(*attList)

In [9]:
clientesRDD3 = clientesRDD2.map(transformToRow)

In [10]:
clientesRDD3.take(3)
clientesRDD3.cache()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [11]:
# Cria DataFrame a partir do RDD
clientesDF = spSession.createDataFrame(clientesRDD3)

In [12]:
len(clientesDF.columns)

370

In [13]:
clientesDF.select("var3", "TARGET").show()

+-----+------+
| var3|TARGET|
+-----+------+
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|  2.0|   0.0|
|229.0|   0.0|
|  2.0|   0.0|
+-----+------+
only showing top 20 rows



## Análise Exploratória

In [14]:
# Classes balanceadas
clientesDF.groupBy("TARGET").count().show()

+------+-----+
|TARGET|count|
+------+-----+
|   0.0|73012|
|   1.0|73012|
+------+-----+



In [15]:
# Verificando alguns dados estatísticos 
clientesDF.describe().toPandas()

,summary,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,ind_var29_0,ind_var29,ind_var30_0,ind_var30,ind_var31_0,ind_var31,ind_var32_cte,ind_var32_0,ind_var32,ind_var33_0,ind_var33,ind_var34_0,ind_var34,ind_var37_cte,ind_var37_0,ind_var37,ind_var39_0,ind_var40_0,ind_var40,ind_var41_0,ind_var41,ind_var39,ind_var44_0,ind_var44,ind_var46_0,ind_var46,num_var1_0,num_var1,num_var4,num_var5_0,num_var5,num_var6_0,num_var6,num_var8_0,num_var8,num_var12_0,num_var12,num_var13_0,num_var13_corto_0,num_var13_corto,num_var13_largo_0,num_var13_largo,num_var13_medio_0,num_var13_medio,num_var13,num_var14_0,num_var14,num_var17_0,num_var17,num_var18_0,num_var18,num_var20_0,num_var20,num_var24_0,num_var24,num_var26_0,num_var26,num_var25_0,num_var25,num_op_var40_hace2,num_op_var40_hace3,num_op_var40_ult1,num_op_var40_ult3,num_op_var41_hace2,num_op_var41_hace3,num_op_var41_ult1,num_op_var41_ult3,num_op_var39_hace2,num_op_var39_hace3,num_op_var39_ult1,num_op_var39_ult3,num_var27_0,num_var28_0,num_var28,num_var27,num_var29_0,num_var29,num_var30_0,num_var30,num_var31_0,num_var31,num_var32_0,num_var32,num_var33_0,num_var33,num_var34_0,num_var34,num_var35,num_var37_med_ult2,num_var37_0,num_var37,num_var39_0,num_var40_0,num_var40,num_var41_0,num_var41,num_var39,num_var42_0,num_var42,num_var44_0,num_var44,num_var46_0,num_var46,saldo_var1,saldo_var5,saldo_var6,saldo_var8,saldo_var12,saldo_var13_corto,saldo_var13_largo,saldo_var13_medio,saldo_var13,saldo_var14,saldo_var17,saldo_var18,saldo_var20,saldo_var24,saldo_var26,saldo_var25,saldo_var28,saldo_var27,saldo_var29,saldo_var30,saldo_var31,saldo_var32,saldo_var33,saldo_var34,saldo_var37,saldo_var40,saldo_var41,saldo_var42,saldo_var44,saldo_var46,var36,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,delta_imp_trasp_var17_out_1y3,delta_imp_trasp_var33_in_1y3,delta_imp_trasp_var33_out_1y3,delta_imp_venta_var44_1y3,delta_num_aport_var13_1y3,delta_num_aport_var17_1y3,delta_num_aport_var33_1y3,delta_num_compra_var44_1y3,delta_num_reemb_var13_1y3,delta_num_reemb_var17_1y3,delta_num_reemb_var33_1y3,delta_num_trasp_var17_in_1y3,delta_num_trasp_var17_out_1y3,delta_num_trasp_var33_in_1y3,delta_num_trasp_var33_out_1y3,delta_num_venta_var44_1y3,imp_amort_var18_hace3,imp_amort_var18_ult1,imp_amort_var34_hace3,imp_amort_var34_ult1,imp_aport_var13_hace3,imp_aport_var13_ult1,imp_aport_var17_hace3,imp_aport_var17_ult1,imp_aport_var33_hace3,imp_aport_var33_ult1,imp_var7_emit_ult1,imp_var7_recib_ult1,imp_compra_var44_hace3,imp_compra_var44_ult1,imp_reemb_var13_hace3,imp_reemb_var13_ult1,imp_reemb_var17_hace3,imp_reemb_var17_ult1,imp_reemb_var33_hace3,imp_reemb_var33_ult1,imp_var43_emit_ult1,imp_trans_var37_ult1,imp_trasp_var17_in_hace3,imp_trasp_var17_in_ult1,imp_trasp_var17_out_hace3,imp_trasp_var17_out_ult1,imp_trasp_var33_in_hace3,imp_trasp_var33_in_ult1,imp_trasp_var33_out_hace3,imp_trasp_var33_out_ult1,imp_venta_var44_hace3,imp_venta_var44_ult1,ind_var7_emit_ult1,ind_var7_recib_ult1,ind_var10_ult1,ind_

In [16]:
# Existem colunas que apresentam valores min e max igual a 0
zeroColumns = ["ind_var2_0", "ind_var2", "ind_var27_0", "ind_var28_0", "ind_var27", "ind_var28",
             "ind_var41", "ind_var46_0", "ind_var46", "num_var27_0", "num_var27", "num_var28_0",
             "num_var28", "num_var41", "num_var46_0", "num_var46", "saldo_var28", "saldo_var27",
             "saldo_var41", "saldo_var46", "imp_amort_var18_hace3", "imp_amort_var34_hace3",
             "imp_reemb_var13_hace3", "imp_reemb_var33_hace3", "imp_trasp_var17_out_hace3",
             "imp_trasp_var33_out_hace3", "num_var2_0_ult1", "num_var2_ult1", "num_reemb_var13_hace3",
             "num_trasp_var17_out_hace3", "num_trasp_var33_out_hace3", "saldo_var2_ult1", 
             "saldo_medio_var13_medio_hace3"]

# Gerando novo DataFrame removendo as colunas
clientesDF = clientesDF.drop(*zeroColumns)

# Pré-processamento

In [18]:
# Função responsável por criar o LabeledPoint, indexar label e retornar um dataframe
def transformToLabeledPoint(row):
    # Utiliza-se row[:-1] para recuperar todas as colunas do DataFrame de forma automática, excluindo a TARGET
    labeledPoint = (row["TARGET"], Vectors.dense(row[:-1]))
    return labeledPoint

# Função responsável por aplicar StringIndexer à variável TARGET
def indexTarget(df):
    stringIndexer = StringIndexer(inputCol = "label", outputCol = "indexed")
    siModel = stringIndexer.fit(df)
    dfTransformed = siModel.transform(df)
    
    return dfTransformed

## Feature Importance

In [19]:
# Função responsável por extrair os nomes das colunas mais relevantes no feature importance
def getFeatureImportancesColumns(featureImportances, dataFrame):
    cols = []
    
    for idx, column in enumerate(dataFrame.columns):
        if idx in featureImportances.indices:
            cols.append(column)
    
    # Inclui coluna TARGET
    cols.append('TARGET')
    
    return cols

In [20]:
# Criando DataFrame com LabeledPoints
clientesRDD4 = clientesDF.rdd.map(transformToLabeledPoint)
df = spSession.createDataFrame(clientesRDD4, ["label", "features"])
# Indexando variável TARGET
clientesDF2 = indexTarget(df)
clientesDF2.cache()

DataFrame[label: double, features: vector, indexed: double]

In [21]:
rf = RandomForestClassifier(numTrees=10, maxDepth=2, labelCol="indexed", featuresCol="features", seed=42,
                           cacheNodeIds = True, subsamplingRate = 0.7)
rfModel = rf.fit(clientesDF2)

In [22]:
rfModel.featureImportances

SparseVector(336, {1: 0.1134, 15: 0.0158, 22: 0.1, 57: 0.1736, 80: 0.0057, 84: 0.0242, 95: 0.0116, 124: 0.0534, 148: 0.0884, 164: 0.0842, 230: 0.0124, 232: 0.0463, 249: 0.0163, 250: 0.0409, 252: 0.0703, 274: 0.014, 293: 0.0537, 299: 0.0758})

In [23]:
# Extraindo variáveis mais relevantes do dataset
colsList = getFeatureImportancesColumns(rfModel.featureImportances, clientesDF)
colsList

['var15',
 'imp_op_var39_efect_ult1',
 'ind_var5',
 'ind_var30',
 'num_var5_0',
 'num_var8_0',
 'num_var13',
 'num_var30_0',
 'saldo_var5',
 'saldo_var30',
 'ind_var9_cte_ult1',
 'ind_var43_emit_ult1',
 'num_var22_ult3',
 'num_med_var22_ult3',
 'num_meses_var5_ult3',
 'num_op_var39_efect_ult3',
 'num_var45_hace3',
 'saldo_medio_var5_ult3',
 'TARGET']

In [24]:
clientesDF3 = clientesDF.select(colsList)
clientesDF3.take(2)

[Row(var15=23.0, imp_op_var39_efect_ult1=0.0, ind_var5=0.0, ind_var30=0.0, num_var5_0=3.0, num_var8_0=0.0, num_var13=0.0, num_var30_0=3.0, saldo_var5=0.0, saldo_var30=0.0, ind_var9_cte_ult1=0.0, ind_var43_emit_ult1=0.0, num_var22_ult3=0.0, num_med_var22_ult3=0.0, num_meses_var5_ult3=0.0, num_op_var39_efect_ult3=0.0, num_var45_hace3=0.0, saldo_medio_var5_ult3=0.0, TARGET=0.0),
 Row(var15=34.0, imp_op_var39_efect_ult1=0.0, ind_var5=0.0, ind_var30=1.0, num_var5_0=3.0, num_var8_0=0.0, num_var13=3.0, num_var30_0=6.0, saldo_var5=0.0, saldo_var30=300.0, ind_var9_cte_ult1=0.0, ind_var43_emit_ult1=0.0, num_var22_ult3=0.0, num_med_var22_ult3=0.0, num_meses_var5_ult3=1.0, num_op_var39_efect_ult3=0.0, num_var45_hace3=0.0, saldo_medio_var5_ult3=0.0, TARGET=0.0)]

In [25]:
# Criando um novo DataFrame com LabeledPoint apenas com as features mais importantes
clientesRDD5 = clientesDF3.rdd.map(transformToLabeledPoint)
df = spSession.createDataFrame(clientesRDD5, ["label", "features"])
clientesFinal = indexTarget(df)
clientesFinal.cache()

DataFrame[label: double, features: vector, indexed: double]

In [26]:
clientesFinal.show()

+-----+--------------------+-------+
|label|            features|indexed|
+-----+--------------------+-------+
|  0.0|[23.0,0.0,0.0,0.0...|    0.0|
|  0.0|[34.0,0.0,0.0,1.0...|    0.0|
|  0.0|[23.0,0.0,1.0,1.0...|    0.0|
|  0.0|[37.0,0.0,1.0,1.0...|    0.0|
|  0.0|[39.0,0.0,0.0,1.0...|    0.0|
|  0.0|[23.0,0.0,0.0,0.0...|    0.0|
|  0.0|[27.0,0.0,1.0,1.0...|    0.0|
|  0.0|[26.0,0.0,1.0,1.0...|    0.0|
|  0.0|[45.0,0.0,0.0,0.0...|    0.0|
|  0.0|[25.0,0.0,1.0,1.0...|    0.0|
|  0.0|[42.0,0.0,0.0,1.0...|    0.0|
|  0.0|[26.0,0.0,0.0,0.0...|    0.0|
|  0.0|[51.0,0.0,1.0,1.0...|    0.0|
|  0.0|[43.0,0.0,0.0,1.0...|    0.0|
|  0.0|[33.0,360.0,0.0,1...|    0.0|
|  0.0|[30.0,0.0,1.0,1.0...|    0.0|
|  0.0|[44.0,0.0,0.0,0.0...|    0.0|
|  0.0|[36.0,0.0,1.0,1.0...|    0.0|
|  0.0|[55.0,0.0,1.0,1.0...|    0.0|
|  0.0|[28.0,0.0,1.0,1.0...|    0.0|
+-----+--------------------+-------+
only showing top 20 rows



## Machine Learning

In [27]:
# Separando em dados de treino e de teste
dados_treino, dados_teste = clientesFinal.randomSplit([0.7, 0.3])

In [28]:
dados_treino.count()

102143

In [29]:
dados_teste.count()

43881

### Gradient-boosted Tree Classifier

In [32]:
gbtClassifier = GBTClassifier(labelCol="indexed", featuresCol="features", maxIter=10)
modelo = gbtClassifier.fit(dados_treino)

In [33]:
# Previsões com os dados de teste
previsoes = modelo.transform(dados_teste)
previsoes.select("features", "indexed", "prediction").show()

+--------------------+-------+----------+
|            features|indexed|prediction|
+--------------------+-------+----------+
|[5.0,0.0,0.0,1.0,...|    0.0|       0.0|
|[5.0,0.0,0.0,1.0,...|    0.0|       0.0|
|[6.0,0.0,0.0,1.0,...|    0.0|       0.0|
|[9.0,0.0,0.0,1.0,...|    0.0|       0.0|
|[10.0,0.0,0.0,1.0...|    0.0|       0.0|
|[11.0,0.0,0.0,1.0...|    0.0|       0.0|
|[12.0,0.0,0.0,1.0...|    0.0|       0.0|
|[13.0,0.0,0.0,1.0...|    0.0|       0.0|
|[14.0,0.0,0.0,0.0...|    0.0|       0.0|
|[14.0,0.0,0.0,1.0...|    0.0|       0.0|
|[14.0,0.0,0.0,1.0...|    0.0|       0.0|
|[15.0,0.0,0.0,0.0...|    0.0|       0.0|
|[15.0,0.0,0.0,1.0...|    0.0|       0.0|
|[15.0,0.0,0.0,1.0...|    0.0|       0.0|
|[16.0,0.0,0.0,0.0...|    0.0|       0.0|
|[16.0,0.0,0.0,0.0...|    0.0|       0.0|
|[17.0,0.0,0.0,1.0...|    0.0|       0.0|
|[19.0,0.0,0.0,1.0...|    0.0|       0.0|
|[21.0,0.0,0.0,0.0...|    0.0|       0.0|
|[21.0,0.0,0.0,1.0...|    0.0|       0.0|
+--------------------+-------+----

## Avaliando o Modelo

In [34]:
def getAcurracy(confusionMatrix):
    TP = confusionMatrix[0][0]
    FN = confusionMatrix[0][1]
    TN = confusionMatrix[1][1]
    FP = confusionMatrix[1][0]

    return (TN + TP / (TN + TP + FN + FP))

# Qual o percentual de casos positivos reais previstos/detectados corretamente
# Mesma fórmula para o Recall - True Positive Rate
def getSensitivity(confusionMatrix):
    TP = confusionMatrix[0][0]
    FN = confusionMatrix[0][1]
    
    return (TP / (TP + FN))

# Qual o percetual de casos negativos reais previstos/detectados incorretamente
def getFalsePositiveRate(confusionMatrix):
    FP = confusionMatrix[1][0]
    TN = confusionMatrix[1][1]
    
    return (FP / (FP + TN))
    
# Qual o percentual de casos negativos reais previstos/detectados corretamente
def getSpecificity(confusionMatrix):
    TN = confusionMatrix[1][1]
    FP = confusionMatrix[1][0]
    
    return (TN / (TN + FP))

# Qual o percentual de previsões positivas estão corretas
def getPrecision(confusionMatrix):
    TP = confusionMatrix[0][0]
    FP = confusionMatrix[1][0]
    
    return (TP / (TP + FP))

# Combinação de Recall e Precision em uma única métrica
def getF1Score(confusionMatrix):
    numerador = 2 * getPrecision(confusionMatrix) * getSensitivity(confusionMatrix) 
    denominador = getPrecision(confusionMatrix) + getSensitivity(confusionMatrix)
    
    return (numerador / denominador)

In [35]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = 'prediction', 
                                              labelCol = 'indexed', 
                                              metricName = 'accuracy')
avaliador.evaluate(previsoes)

0.8907044050955994

In [36]:
# previsoesRDD = (previsões, label)
previsoesRDD = previsoes.select("label", "prediction").rdd.map(lambda line: (line[1], line[0]))

In [37]:
metrics = MulticlassMetrics(previsoesRDD)

In [38]:
# Classes atuais estão nas linhas
# Classes previstas estão nas colunas
#           0    1   
# Atual 0 [TP | FN]
# Atual 1 [FP | TN]
metrics.confusionMatrix().toArray()

array([[19682.,  2213.],
       [ 2583., 19403.]])

In [39]:
# Confusion Matrix
previsoes.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|19403|
|  0.0|       1.0| 2213|
|  1.0|       0.0| 2583|
|  0.0|       0.0|19682|
+-----+----------+-----+



In [40]:
#Precision and recall are often in tension. That is, improving precision typically reduces recall and vice versa.

print(getSensitivity(metrics.confusionMatrix().toArray()))
print(getSpecificity(metrics.confusionMatrix().toArray()))

0.8989266955926011
0.8825161466387701


## Considerações Finais

- O modelo apresentou uma acurácia de 89%, sendo acima da métrica estabelecida inicialmente
- A especificidade do modelo atingiu 88%, isto indica que o modelo prevê de forma satisfatória clientes insatisfeitos, sendo este o objetivo inicial do projeto
- Por fim, o modelo também apresentou 89% de sensitividade, indicando que também prevê de forma satisfatória clientes satisfeitos